In [2]:
! pip install datasets transformers seqeval

In [4]:
pip install fastprogress

Note: you may need to restart the kernel to use updated packages.


In [71]:
from pathlib import Path
from argparse import Namespace
from typing import Union, List
from fastprogress import progress_bar
from typing_extensions import TypedDict
import torch
from datasets import Dataset, DatasetDict, load_metric
import numpy as np
from transformers import DataCollatorForTokenClassification, AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import HerbertTokenizer, RobertaModel


In [41]:
data_dir = Path("/Users/peiwentang/Desktop/2021Fall/nyu-nlp-final-project/feature_extraction/")

args = Namespace(
    batch_size=64,
    num_workers=4
)

train_data_path = data_dir / '%-training_new'
valid_data_path = data_dir / '%-dev_new'
test_data_path = data_dir / '%-test_new'

In [42]:
SYMBOL_DICT = {
    "COMMA": ",",
}

LABEL_LIST = ["NONE", "PRED", "ARG1", "SUPPORT", "NOARG"]
POS_LIST = ["CC", "CD", "DT", "FW", "IN", "JJ", "JJR", "JJS", 
            "LS", "MD", "NN", "NNS", "NNP", "NNPS", "PDT", 
            "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "SYM", 
            "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP",
            "VBZ", "WDT", "WP", "WP$", "WRB", "PU", "EX", 
            "RP"]
BIO_TAG_CONVERSION_DICT = {
    ".": "PU",
    ",": "PU",
    "COMMA": "PU",
    "$": "PU",
    ":": "PU",
    "(": "PU",
    ")": "PU",
    "``": "PU",
    "''": "PU",
    "#": "PU"
}
BIO_TAG_LIST = ["O", "B-NP", "I-NP", "B-VP", "I-VP", "B-PP",
                "I-PP", "B-ADJP", "I-ADJP", "B-ADVP", "I-ADVP",
                "B-SBAR", "I-SBAR", "B-PRT", "I-PRT", "B-CONJP",
                "I-CONJP", "B-UCP", "I-UCP"]

In [43]:
class Word(TypedDict):
  word: str
  pos: str
  biotag: str
  word2vec: float
  label: Union[str, None]

def parse_input(input_file: Union[str, Path], drop_label = False) -> List[Union[List[Word], None]]:
    """
    Parses the input file and returns a list of lists of words.
    """
    with open(input_file, "r") as f:
        lines = f.readlines()
    sentences: List[Union[List[Word], None]] = []
    last_sentence: List[Word] = []
    print("Parsing input file lines...")
    line_no = 0
    for line in progress_bar(lines):
        line_no += 1
        line = line.strip()
        word_info = line.split("\t")
        if len(word_info) >= 6:
            word_str = word_info[0].strip()
            if word_str in SYMBOL_DICT:
              word_str = SYMBOL_DICT[word_str]
            pos = word_info[1].strip()
            if pos in BIO_TAG_CONVERSION_DICT:
              pos = BIO_TAG_CONVERSION_DICT[pos]
            if pos not in POS_LIST:
              print(f"Warning: invalid POS on line {line_no} \"{pos}\", treated as PU.")
              pos = "PU"
            biotag = word_info[2].strip()
            if biotag not in BIO_TAG_LIST:
              print(f"Warning: invalid bio tag on line {line_no} \"{biotag}\", treated as O.")
              biotag = "O"
            if len(word_info) >= 6:
                label = word_info[5].strip()
            else:
                label = "NONE"
            if label not in LABEL_LIST:
              print(f"Warning: invalid label on line {line_no} \"{label}\", treated as NONE.")
              label = "NONE"
            if drop_label:
              label = None
            word2vec = word_info[6]
            word = Word(word=word_str, pos=pos, biotag=biotag, label=label, word2vec=word2vec)
            last_sentence.append(word)
        else:
            if len(last_sentence) > 0:
                sentences.append(last_sentence)
            last_sentence = []
    if len(last_sentence) > 0:
        sentences.append(last_sentence)
    return sentences

In [44]:
train_sentences = parse_input(train_data_path)
valid_sentences = parse_input(valid_data_path)
test_sentences = parse_input(test_data_path, drop_label=True)

Parsing input file lines...


Parsing input file lines...


Parsing input file lines...


In [65]:
pip install sentencepiece

     |████████████████████████████████| 1.1 MB 49 kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [110]:
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained("roberta-base", num_labels=len(LABEL_LIST))

loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at /Users/peiwentang/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at /Users/peiwentang/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer.json from cache at /Users/peiwentang/.cache/huggingface/transformers/d53fc0fa09b8342651efd4073d75e19617b3e51287c2a535becda5808a8db287.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache

In [105]:
def build_dataset_from_sentences(sentences, drop_label = False):
  dataset_tokens = []
  dataset_partitive_roles = []
  dataset_pos_tags = []
  dataset_bio_tags = []
  dataset_word2vec = []
  for sentence in sentences:
    tokens = [word['word'] for word in sentence]
    if not drop_label:
      partitive_roles = [LABEL_LIST.index(word['label']) for word in sentence]
    else:
      partitive_roles = None
    pos_tags = [POS_LIST.index(word['pos']) for word in sentence]
    bio_tags = [BIO_TAG_LIST.index(word['biotag']) for word in sentence]
    word2vec = [word['word2vec'] for word in sentence]
    dataset_tokens.append(tokens)
    if not drop_label:
      dataset_partitive_roles.append(partitive_roles)
    dataset_pos_tags.append(pos_tags)
    dataset_bio_tags.append(bio_tags)
    dataset_word2vec.append(word2vec)
  if not drop_label:
    dataset_dict = {
        "tokens": dataset_tokens,
        "partitive_roles": dataset_partitive_roles,
        "pos_tags": dataset_pos_tags,
        "bio_tags": dataset_bio_tags,
        "word2vec": dataset_word2vec
    }
  else:
    dataset_dict = {
        "tokens": dataset_tokens,
        "pos_tags": dataset_pos_tags,
        "bio_tags": dataset_bio_tags,
        "word2vec": dataset_word2vec
    }
  return Dataset.from_dict(dataset_dict)

In [106]:
train_raw_dataset = build_dataset_from_sentences(train_sentences)
valid_raw_dataset = build_dataset_from_sentences(valid_sentences)
test_raw_dataset = build_dataset_from_sentences(test_sentences, drop_label=True)

raw_datasets = DatasetDict(train=train_raw_dataset, valid=valid_raw_dataset, test=test_raw_dataset)

In [107]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    if "partitive_roles" in examples:
      labels = []
      for i, label in enumerate(examples["partitive_roles"]):
          word_ids = tokenized_inputs.word_ids(batch_index=i)
          previous_word_idx = None
          label_ids = []
          for word_idx in word_ids:
              # Special tokens have a word id that is None. We set the label to -100 so they are automatically
              # ignored in the loss function.
              if word_idx is None:
                  label_ids.append(-100)
              # We set the label for the first token of each word.
              elif word_idx != previous_word_idx:
                  label_ids.append(label[word_idx])
              # For the other tokens in a word, we set the label to either the current label or -100, depending on
              # the label_all_tokens flag.
              else:
                  label_ids.append(label[word_idx] if label_all_tokens else -100)
              previous_word_idx = word_idx

          labels.append(label_ids)

      tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [111]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [116]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [LABEL_LIST[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [LABEL_LIST[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [117]:
batch_size=64

train_args = TrainingArguments(
    "bert_partitive_roles",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01
)

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [118]:
trainer = Trainer(
    model,
    train_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [119]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, pos_tags, partitive_roles, bio_tags, word2vec. If tokens, pos_tags, partitive_roles, bio_tags, word2vec are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2367
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 370


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.071507,0.773512,0.815789,0.794089,0.970183
2,No log,0.052084,0.823409,0.811741,0.817533,0.975153
3,No log,0.055528,0.759717,0.870445,0.811321,0.970948
4,No log,0.049670,0.830612,0.823887,0.827236,0.975153
5,No log,0.049013,0.834356,0.825911,0.830112,0.975535
6,No log,0.048723,0.831301,0.827935,0.829615,0.975535
7,No log,0.048153,0.824701,0.838057,0.831325,0.975535
8,No log,0.048217,0.824111,0.844130,0.834000,0.975917
9,No log,0.048392,0.821569,0.848178,0.834661,0.975917
10,No log,0.048125,0.831325,0.838057,0.834677,0.975917


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, pos_tags, partitive_roles, bio_tags, word2vec. If tokens, pos_tags, partitive_roles, bio_tags, word2vec are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 83
  Batch size = 64
/Users/peiwentang/anaconda3/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOARG seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/peiwentang/anaconda3/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PRED seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/Users/peiwentang/anaconda3/lib/python3.8/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SUPPORT seems not to be NE tag.
  warnings.warn('{} seems not 

TrainOutput(global_step=370, training_loss=0.04460271886877112, metrics={'train_runtime': 13940.724, 'train_samples_per_second': 1.698, 'train_steps_per_second': 0.027, 'total_flos': 824868867605640.0, 'train_loss': 0.04460271886877112, 'epoch': 10.0})

In [120]:
test_results = trainer.predict(tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, bio_tags, pos_tags, word2vec. If tokens, bio_tags, pos_tags, word2vec are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 150
  Batch size = 64


In [121]:
np.argmax([1, 5, 3, 5])

1

In [122]:
out = []

for i in range(len(tokenized_datasets['test'])):
  sentence = tokenized_datasets['test'][i]
  tokenized_input = tokenizer(sentence["tokens"], truncation=True, is_split_into_words=True)
  predictions = test_results.predictions[i]
  label_ids = []
  for prediction in predictions:
    label_ids.append(np.argmax(prediction))
  word_id_to_label_idx = {}
  for j, word_id in enumerate(tokenized_input.word_ids()):
    if word_id in word_id_to_label_idx or word_id is None:
      continue
    word_id_to_label_idx[word_id] = j
  labelings = []
  for j, token in enumerate(sentence["tokens"]):
    label_idx = word_id_to_label_idx[j]
    label_id = label_ids[label_idx]
    label = LABEL_LIST[label_id] if label_id != 0 else None
    labelings.append((token, label))
  out.append(labelings)


In [123]:
out[0][:10]

[('Then', 'NOARG'),
 ('in', 'NOARG'),
 ('a', 'NOARG'),
 ('lightning', 'NOARG'),
 ('plunge', 'NOARG'),
 (',', 'NOARG'),
 ('the', 'NOARG'),
 ('Dow', 'NOARG'),
 ('Jones', 'NOARG'),
 ('industrials', 'NOARG')]

In [124]:
out_path = data_dir / 'test-out2'

In [125]:
with open(out_path, 'w') as f:
  for line in out:
    for labling in line:
      if labling[1]:
        f.write(f"{labling[0]}\t{labling[1]}\n")
      else:
        f.write(f"{labling[0]}\n")
    f.write("\n")

In [126]:
def test_sentence_string(s: str):
  tokenized_input = tokenizer(s, truncation=True)
  model(tokenized_input)